In [ ]:
import os
import nibabel as nib
import numpy as np
import pandas as pd
import cortex
np.random.seed(1234)
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams.update({'font.size': 13})

subject = '2009c'
xfm = 'fullhead_v64'

In [ ]:
base = '/users/jmy/data/image_sound'
sbjs_num = 15
sbj_list = [25,26,29,30,31,32,33,34,37,38,39,40,41,43,44]
type_, stim_type = 'Vs', 'etr'
depth = 15
affine = np.array([[   3. ,   -0. ,   -0. ,  -94.5],
       [  -0. ,    3. ,   -0. , -130.5],
       [   0. ,    0. ,    3. ,  -76.5],
       [   0. ,    0. ,    0. ,    1. ]])

In [ ]:
def save_nifti(vol, savepath):
    img1 = nib.Nifti1Image(vol,affine=affine)
    nib.save(img1,savepath)

In [ ]:
# corrected values using random permutation
tpthr = 0.05
sv_key = 'PCA3_90'
savebase = base+'/RSA_Wholebrain_CNN/'+sv_key
# Grp
ci = 3
type_, stim_type = interest_list[ci], stim_type_list[ci]
tmpvols = np.zeros((depth,64,76,64))
ptmpvols = np.zeros((depth,64,76,64))
for li in range(depth):
    loadpath = savebase+'/Group/corr5e-02_tmap_'+type_+'_'+stim_type+'_L'+str(li+1)+'.nii'
    tmpvols[li] = nib.load(loadpath).get_fdata()
# exclude all neg
tmpsum = np.sum((tmpvols>0),axis=0)
tmpsum2 = np.sum((tmpvols<0),axis=0)
msk_idx = np.where((tmpsum > 0)&(tmpsum2!=15))
max_info = np.argmax(tmpvols.transpose(1,2,3,0)[msk_idx],axis=1) # vec,depth
max_info += 1
# wholebrain
assignvol = np.zeros((64,76,64))
assignvol[msk_idx] = max_info
savepath = savebase+'/Group/corr5e-02_assgn_'+type_+'_'+stim_type+'.nii'
save_nifti(assignvol,savepath)

In [ ]:
savebase = base+'/For_thesis/Corrected'
os.chdir(savebase)

In [ ]:
dir_info = ['PCA3_90','PCA1_90']
di = 0

In [ ]:
# Make individual layer cluster nifti file
path = base+'/RSA_Wholebrain_CNN/PCA3_90/Group/corr5e-02_assgn_'+type_+'_'+stim_type+'.nii'
tmpvol = nib.load(path).get_fdata()
for li in range(15):
    tmpvol2 = np.zeros((64,76,64))
    tmpvol2[np.where(tmpvol == li+1)] = 1
    img1 = nib.Nifti1Image(tmpvol2,affine)
    nib.save(img1,'/users/jmy/data/image_sound/For_thesis/Corrected/'+str(di).zfill(2)+'_'+type_+str(li+1)+'.nii')

In [ ]:
# 3dclusterize size 20
for li in range(depth):
    cmd = '3dClusterize -nosum -1Dformat -inset {}_{}{}.nii -idat 0 -ithr 0 -NN 2 -clust_nvox 20 -bisided -0.0001 0.0001 -pref_map {}_{}{}_clst.nii -overwrite'.format(str(di).zfill(2),type_,li+1,str(di).zfill(2),type_,li+1)
    os.system(cmd)

### Cluster map to find the location

In [ ]:
vol2sv = np.zeros((64,76,64))#clst_in_one_10.nii
for li in range(depth):
    filename = '{}_{}{}_clst.nii'.format(str(di).zfill(2),type_,li+1)
    if os.path.exists(filename):
        tmp = nib.load(filename).get_fdata()
        vol2sv[np.where(tmp>0)] = li+1
vol2sv[(vol2sv==0)] = np.nan
img1 = nib.Nifti1Image(vol2sv,affine)
savepath = str(di).zfill(2)+'_clst_in_one_vthr15.nii'
nib.save(img1,savepath)

In [ ]:
# corrected t-score load
tmaps = np.zeros((15,64,76,64))
for li2 in range(depth):
    loadpath = base+'/RSA_Wholebrain_CNN/PCA3_90/Group/corr5e-02_tmap_'+type_+'_'+stim_type+'_L'+str(li2+1)+'.nii'
    tmaps[li2] = nib.load(loadpath).get_fdata()

In [ ]:
# uncorrected
tmaps_uncorr = np.zeros((15,64,76,64))
for li2 in range(depth):
    loadpath = base+'/RSA_Wholebrain_CNN/PCA3_90/Group/tmap_'+type_+'_'+stim_type+'_L'+str(li2+1)+'.nii'
    tmaps_uncorr[li2] = nib.load(loadpath).get_fdata()

# Plot one by one

In [ ]:
from pathlib import Path
import matplotlib as mpl
fpath = Path(mpl.get_data_path(), "/users/jmy/data/image_sound/arial.ttf")
sns.set(style='dark',font_scale=1.6)
col_list = sns.color_palette('Spectral_r',15)
col_arr = np.array(col_list)
xpos = np.arange(15)
xlabels=['1','2','3','4','5','6','7','8','9','10','11','12','13','Fc','Out\nput']

In [ ]:
# w.o. title
def make_bar_graph(mli, clstn, ylim=None):
    clst_ = nib.load('{}_{}{}_clst.nii'.format(str(di).zfill(2),type_,mli)).get_fdata()
    clst_idx = np.where(clst_== clstn)
    lay_mean = np.zeros((15))
    lay_ste = np.zeros((15))
    sig_info = np.ones((15))
    for li2 in range(depth):
        tmap = tmaps[li2] 
        vec = tmap[clst_idx]
        if np.sum(vec) == 0:
            vec = tmaps_uncorr[li2][clst_idx]
            sig_info[li2] = 0
        lay_mean[li2] = vec.mean()
        lay_ste[li2] = vec.std()/vec.shape[0]**0.5
    cor_idx = np.where(sig_info==1)
    tran_idx = np.where(sig_info!=1)
    fig,axes = plt.subplots(1,1,figsize=(8,5),facecolor='white',constrained_layout=True)
    axes.bar(xpos[cor_idx],lay_mean[cor_idx],color=col_arr[cor_idx],width=0.9,linewidth=1.5,
            yerr=lay_ste[cor_idx],error_kw=dict(lw=1.2),ecolor='k',capsize=6)
    axes.bar(xpos[tran_idx],lay_mean[tran_idx],color='None',edgecolor='grey',width=0.9,linewidth=1.0,
            yerr=lay_ste[tran_idx],error_kw=dict(lw=1.2),ecolor='k',capsize=6)
    if ylim != None:
        axes.set_yticks(np.arange(ylim))
    axes.set_xticks(xpos)
    axes.set_xticklabels(xlabels)
    axes.set(xlabel='CNN layer', ylabel="t-score")
    filename = 'Layer'+str(mli)+'_clst'+str(clstn)+'_mod_wott.jpg'
    if ylim != None:
        filename = 'ylim'+str(ylim)+'_'+filename
    fig.savefig(base+'/For_thesis/Corrected/Rslt/New_Labels/'+filename)
    plt.close()

In [ ]:
for mli in range(1,16):
    clst_ = nib.load('{}_{}{}_clst.nii'.format(str(di).zfill(2),type_,mli)).get_fdata()
    totn = int(clst_.max())
    for clstn in range(1,totn+1):
        make_bar_graph(mli,clstn,ylim=6.0)

# Average

In [ ]:
def make_bar_graph_avg(mli, ylim=None):
    clst_ = nib.load('{}_{}{}_clst.nii'.format(str(di).zfill(2),type_,mli)).get_fdata()
    totn = int(clst_.max())
    arr2avg = np.zeros((totn,15))
    sig_info4avg = np.ones((totn,15))
    roiname = ''
    for clstn in range(1,totn+1):
        clst_idx = np.where(clst_== clstn)
        for li2 in range(depth):
            tmap = tmaps[li2] 
            vec = tmap[clst_idx]
            if np.sum(vec) == 0:
                vec = tmaps_uncorr[li2][clst_idx]
                sig_info4avg[clstn-1,li2] = 0
            arr2avg[clstn-1,li2] = vec.mean()
    lay_mean = np.zeros((15))
    lay_ste = np.zeros((15))
    sig_info = np.ones((15))
    for li2 in range(depth):
        tmparr = arr2avg[:,li2]
        cor_idx = np.where(sig_info4avg[:,li2]==1) # only significant cluster
        tran_idx = np.where(sig_info4avg[:,li2]!=1)
        vectmp = tmparr[cor_idx]
        if vectmp.shape[0] == 0:
            vectmp = tmparr[tran_idx]
            sig_info[li2] = 0
        lay_mean[li2] = np.mean(vectmp)
        lay_ste[li2] = np.std(vectmp)/vectmp.shape[0]**0.5
    cor_idx = np.where(sig_info==1)
    tran_idx = np.where(sig_info!=1)    
    fig, axes = plt.subplots(1,1,figsize=(8,5),facecolor='white',constrained_layout=True)
    if totn == 1:
        axes.bar(xpos[cor_idx],lay_mean[cor_idx],color=col_arr[cor_idx],width=0.9,linewidth=1.5)
        axes.bar(xpos[tran_idx],lay_mean[tran_idx],color='None',edgecolor='grey',width=0.9)
    else:
        axes.bar(xpos[cor_idx],lay_mean[cor_idx],color=col_arr[cor_idx],width=0.9,linewidth=1.5,
                            yerr=lay_ste[cor_idx],error_kw=dict(lw=1.2),ecolor='k',capsize=6)
        axes.bar(xpos[tran_idx],lay_mean[tran_idx],color='None',edgecolor='grey',width=0.9,linewidth=1.0,
                    yerr=lay_ste[tran_idx],error_kw=dict(lw=1.2),ecolor='k',capsize=6)
    if ylim != None:
        axes.set_yticks(np.arange(ylim))
    axes.set_xticks(xpos)
    axes.set_xticklabels(xlabels, fontsize=20)
    axes.set_xlabel('CNN Layer', font=fpath, fontsize=25)
    axes.set_ylabel('t-score', font=fpath, fontsize=25)
    filename = 'Avgd_Layer'+str(mli)+'.jpg'
    if ylim != None:
        filename = 'fsize_ylim'+str(ylim)+'_'+filename
    fig.savefig(base+'/For_thesis/Corrected/Rslt/New_Labels/'+filename)
    plt.close()

In [ ]:
for mli in range(1,16):
    if mli <14:
        ylim = 6.0
    else:
        ylim = 4.0
    make_bar_graph_avg(mli, ylim=ylim)

# Figure - initial, middle, later, FC-Out

In [ ]:
num2ass = [1,3,4,5,6,7,8,9,10,11,12,13,14,15,16]

In [ ]:
vols = {}
vol2sv = np.zeros((64,76,64))
for li in [0,1]:#,2,3]:
    filename = '{}_{}{}_clst.nii'.format(str(di).zfill(2),type_,li+1)
    tmp = nib.load(filename).get_fdata()
    vol2sv[np.where(tmp>0)] = num2ass[li]#li+1
vol2sv[(vol2sv==0)] = np.nan
img1 = nib.Nifti1Image(vol2sv,affine)
savepath = str(li)+'.nii'
nib.save(img1,savepath)
vols['Initial'] = cortex.Volume(savepath, subject, xfm, vmin=1,vmax=16,cmap='jet')

In [ ]:
vol2sv = np.zeros((64,76,64))
for li in np.arange(2,10):
    filename = '{}_{}{}_clst.nii'.format(str(di).zfill(2),type_,li+1)
    tmp = nib.load(filename).get_fdata()
    vol2sv[np.where(tmp>0)] = num2ass[li]#li+1
vol2sv[(vol2sv==0)] = np.nan
img1 = nib.Nifti1Image(vol2sv,affine)
savepath = str(li)+'.nii'
nib.save(img1,savepath)
vols['Middle'] = cortex.Volume(savepath, subject, xfm, vmin=1,vmax=16,cmap='jet')

In [ ]:
vol2sv = np.zeros((64,76,64))
for li in np.arange(10,13):
    filename = '{}_{}{}_clst.nii'.format(str(di).zfill(2),type_,li+1)
    tmp = nib.load(filename).get_fdata()
    vol2sv[np.where(tmp>0)] = num2ass[li]#li+1
vol2sv[(vol2sv==0)] = np.nan
img1 = nib.Nifti1Image(vol2sv,affine)
savepath = str(li)+'.nii'
nib.save(img1,savepath)
vols['Later'] = cortex.Volume(savepath, subject, xfm, vmin=1,vmax=16,cmap='jet')

In [ ]:
vol2sv = np.zeros((64,76,64))
for li in np.arange(13,15):
    filename = '{}_{}{}_clst.nii'.format(str(di).zfill(2),type_,li+1)
    tmp = nib.load(filename).get_fdata()
    vol2sv[np.where(tmp>0)] = num2ass[li]#li+1
vol2sv[(vol2sv==0)] = np.nan
img1 = nib.Nifti1Image(vol2sv,affine)
savepath = str(li)+'.nii'
nib.save(img1,savepath)
vols['Cate'] = cortex.Volume(savepath, subject, xfm, vmin=1,vmax=16,cmap='jet')

In [ ]:
#Assigned
savepath = str(di).zfill(2)+'_clst_in_one_vthr15.nii'
vol2sv = nib.load(savepath).get_fdata()
for li in np.arange(2,16)[::-1]:
    vol2sv[(vol2sv==li)] = li+1
img1 = nib.Nifti1Image(vol2sv,affine)
savepath = 'assgn.nii'
nib.save(img1,savepath)
vols['Assgn'] = cortex.Volume(savepath, subject, xfm, vmin=1, vmax=16,cmap='jet')

In [ ]:
cortex.webgl.show(vols)#,overlay_file='/users/jmy/data/pycortex_overlays/overlays_ICA_final_5.svg',recache=True)